#**Individual Practical Project**
Course: MSc Data Science

Subject: Applied Machine Learning

Cyber Threat Detection Competition Using Machine Learning

AssignmentType: Individual Practical Project

Dataset: Aegean WiFi Intrusion Dataset (AWID)




###**Downloading Dataset**

In [ ]:
# Attempt to make the project reproducible
import random

random.seed(101)
print(random.random())

0.5811521325045647


In [ ]:
#Loading the AWID dataset, adding headers and loading back to disk
import pandas as pd

headername = 'headers_without4and7.csv'
headers = pd.read_csv(headername)

filename = 'train_imperson_without4n7_balanced_data.csv' 
testfilename = 'test_imperson_without4n7_balanced_data.csv'

training = pd.read_csv(filename)
testing = pd.read_csv(testfilename)

training.columns = headers.columns
testing.columns = headers.columns

training.to_csv('train_data_with_headers_without4n7_balanced_data.csv' , sep=',' , index=False)
testing.to_csv('test_data_with_headers_without4n7_balanced_data.csv' , sep=',' , index=False)


In [ ]:
print(training.shape)
training.head()

(97044, 153)


,frame.interface_id,frame.dlt,frame.offset_shift,frame.time_delta,frame.time_delta_displayed,frame.len,frame.cap_len,frame.marked,frame.ignored,radiotap.version,radiotap.pad,radiotap.length,radiotap.present.tsft,radiotap.present.flags,radiotap.present.rate,radiotap.present.channel,radiotap.present.fhss,radiotap.present.dbm_antsignal,radiotap.present.dbm_antnoise,radiotap.present.lock_quality,radiotap.present.tx_attenuation,radiotap.present.db_tx_attenuation,radiotap.present.dbm_tx_power,radiotap.present.antenna,radiotap.present.db_antsignal,radiotap.present.db_antnoise,radiotap.present.rxflags,radiotap.present.xchannel,radiotap.present.mcs,radiotap.present.ampdu,radiotap.present.vht,radiotap.present.reserved,radiotap.present.rtap_ns,radiotap.present.vendor_ns,radiotap.present.ext,radiotap.mactime,radiotap.flags.cfp,radiotap.flags.preamble,radiotap.flags.wep,radiotap.flags.frag,...,wlan_mgt.fixed.fragment,wlan_mgt.fixed.sequence,wlan_mgt.tagged.all,wlan_mgt.ssid,wlan_mgt.ds.current_channel,wlan_mgt.tim.dtim_count,wlan_mgt.tim.dtim_period,wlan_mgt.tim.bmapctl.multicast,wlan_mgt.tim.bmapctl.offset,wlan_mgt.country_info.environment,wlan_mgt.rsn.version,wlan_mgt.rsn.gcs.type,wlan_mgt.rsn.pcs.count,wlan_mgt.rsn.akms.count,wlan_mgt.rsn.akms.type,wlan_mgt.rsn.capabilities.preauth,wlan_mgt.rsn.capabilities.no_pairwise,wlan_mgt.rsn.capabilities.ptksa_replay_counter,wlan_mgt.rsn.capabilities.gtksa_replay_counter,wlan_mgt.rsn.capabilities.mfpr,wlan_mgt.rsn.capabilities.mfpc,wlan_mgt.rsn.capabilities.peerkey,wlan_mgt.tcprep.trsmt_pow,wlan_mgt.tcprep.link_mrg,wlan.wep.iv,wlan.wep.key,wlan.wep.icv,wlan.tkip.extiv,wlan.ccmp.extiv,wlan.qos.tid,wlan.qos.priority,wlan.qos.eosp,wlan.qos.ack,wlan.qos.amsdupresent,wlan.qos.buf_state_indicated,wlan.qos.bit4,wlan.qos.txop_dur_req,wlan.qos.buf_state_indicated.1,data.len,class
0,0,0,0,0.000066,0.000066,0.009150,0.009150,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.36865,0,0,0,0,...,0,0.0,0,0.000000,0.00000,0.000000,0.000000,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0
1,0,0,0,0.000014,0.000014,0.000000,0.000000,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.36867,0,0,0,0,...,0,0.0,0,0.000000,0.00000,0.000000,0.000000,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0
2,0,0,0,0.035528,0.035528,0.070588,0.070588,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.36871,0,0,0,0,...,0,0.0,1,0.001459,0.46154,0.000000,0.007936,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0
3,0,0,0,0.005128,0.005128,0.094771,0.094771,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.36876,0,0,0,0,...,0,0.0,1,0.000875,0.46154,0.000000,0.003968,0,0,0.0,0.000015,0.007843,0.000031,0.000016,1,0,0,0,0,0,0,0,0.5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0
4,0,0,0,0.035116,0.035116,0.070588,0.070588,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.36880,0,0,0,0,...,0,0.0,1,0.001459,0.46154,0.004566,0.007936,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0


In [ ]:
print(testing.shape)
testing.head()

(40158, 153)


,frame.interface_id,frame.dlt,frame.offset_shift,frame.time_delta,frame.time_delta_displayed,frame.len,frame.cap_len,frame.marked,frame.ignored,radiotap.version,radiotap.pad,radiotap.length,radiotap.present.tsft,radiotap.present.flags,radiotap.present.rate,radiotap.present.channel,radiotap.present.fhss,radiotap.present.dbm_antsignal,radiotap.present.dbm_antnoise,radiotap.present.lock_quality,radiotap.present.tx_attenuation,radiotap.present.db_tx_attenuation,radiotap.present.dbm_tx_power,radiotap.present.antenna,radiotap.present.db_antsignal,radiotap.present.db_antnoise,radiotap.present.rxflags,radiotap.present.xchannel,radiotap.present.mcs,radiotap.present.ampdu,radiotap.present.vht,radiotap.present.reserved,radiotap.present.rtap_ns,radiotap.present.vendor_ns,radiotap.present.ext,radiotap.mactime,radiotap.flags.cfp,radiotap.flags.preamble,radiotap.flags.wep,radiotap.flags.frag,...,wlan_mgt.fixed.fragment,wlan_mgt.fixed.sequence,wlan_mgt.tagged.all,wlan_mgt.ssid,wlan_mgt.ds.current_channel,wlan_mgt.tim.dtim_count,wlan_mgt.tim.dtim_period,wlan_mgt.tim.bmapctl.multicast,wlan_mgt.tim.bmapctl.offset,wlan_mgt.country_info.environment,wlan_mgt.rsn.version,wlan_mgt.rsn.gcs.type,wlan_mgt.rsn.pcs.count,wlan_mgt.rsn.akms.count,wlan_mgt.rsn.akms.type,wlan_mgt.rsn.capabilities.preauth,wlan_mgt.rsn.capabilities.no_pairwise,wlan_mgt.rsn.capabilities.ptksa_replay_counter,wlan_mgt.rsn.capabilities.gtksa_replay_counter,wlan_mgt.rsn.capabilities.mfpr,wlan_mgt.rsn.capabilities.mfpc,wlan_mgt.rsn.capabilities.peerkey,wlan_mgt.tcprep.trsmt_pow,wlan_mgt.tcprep.link_mrg,wlan.wep.iv,wlan.wep.key,wlan.wep.icv,wlan.tkip.extiv,wlan.ccmp.extiv,wlan.qos.tid,wlan.qos.priority,wlan.qos.eosp,wlan.qos.ack,wlan.qos.amsdupresent,wlan.qos.buf_state_indicated,wlan.qos.bit4,wlan.qos.txop_dur_req,wlan.qos.buf_state_indicated.1,data.len,class
0,0,0,0,0.002547,0.002547,0.98824,0.98824,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.88219,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.22215,0.0,0.47729,0.0,0.0,0.16667,0.16667,0,0,0,0,0,0,0,0.98674,0
1,0,0,0,0.003296,0.003296,0.98824,0.98824,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.88220,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.35106,0.0,0.18516,0.0,0.0,0.16667,0.16667,0,0,0,0,0,0,0,0.98674,0
2,0,0,0,0.003285,0.003285,0.98824,0.98824,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.88220,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.43700,0.0,0.47541,0.0,0.0,0.16667,0.16667,0,0,0,0,0,0,0,0.98674,0
3,0,0,0,0.005942,0.005942,0.00000,0.00000,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.88220,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.00000,0.0,0.00000,0.0,0.0,0.00000,0.00000,0,0,0,0,0,0,0,0.00000,0
4,0,0,0,0.001519,0.001519,0.98824,0.98824,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.88220,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.63621,0.0,0.89971,0.0,0.0,0.16667,0.16667,0,0,0,0,0,0,0,0.98674,0


In [ ]:
# Divide dataset with and without the target variable

# Dataset without the target variable (so last column has been removed) for training dataset
x_training = training.iloc[:,0:152] 

# Target variable for training dataset
y_training = training.iloc[:,152] 

# Dataset without the target variable (so last column has been removed) for testing dataset
x_testing = testing.iloc[:,0:152]

# Target variable for testing dataset
y_testing = testing.iloc[:,152]

In [ ]:
print(x_training.shape)
x_training.head()

(97044, 152)


,frame.interface_id,frame.dlt,frame.offset_shift,frame.time_delta,frame.time_delta_displayed,frame.len,frame.cap_len,frame.marked,frame.ignored,radiotap.version,radiotap.pad,radiotap.length,radiotap.present.tsft,radiotap.present.flags,radiotap.present.rate,radiotap.present.channel,radiotap.present.fhss,radiotap.present.dbm_antsignal,radiotap.present.dbm_antnoise,radiotap.present.lock_quality,radiotap.present.tx_attenuation,radiotap.present.db_tx_attenuation,radiotap.present.dbm_tx_power,radiotap.present.antenna,radiotap.present.db_antsignal,radiotap.present.db_antnoise,radiotap.present.rxflags,radiotap.present.xchannel,radiotap.present.mcs,radiotap.present.ampdu,radiotap.present.vht,radiotap.present.reserved,radiotap.present.rtap_ns,radiotap.present.vendor_ns,radiotap.present.ext,radiotap.mactime,radiotap.flags.cfp,radiotap.flags.preamble,radiotap.flags.wep,radiotap.flags.frag,...,wlan_mgt.fixed.chanwidth,wlan_mgt.fixed.fragment,wlan_mgt.fixed.sequence,wlan_mgt.tagged.all,wlan_mgt.ssid,wlan_mgt.ds.current_channel,wlan_mgt.tim.dtim_count,wlan_mgt.tim.dtim_period,wlan_mgt.tim.bmapctl.multicast,wlan_mgt.tim.bmapctl.offset,wlan_mgt.country_info.environment,wlan_mgt.rsn.version,wlan_mgt.rsn.gcs.type,wlan_mgt.rsn.pcs.count,wlan_mgt.rsn.akms.count,wlan_mgt.rsn.akms.type,wlan_mgt.rsn.capabilities.preauth,wlan_mgt.rsn.capabilities.no_pairwise,wlan_mgt.rsn.capabilities.ptksa_replay_counter,wlan_mgt.rsn.capabilities.gtksa_replay_counter,wlan_mgt.rsn.capabilities.mfpr,wlan_mgt.rsn.capabilities.mfpc,wlan_mgt.rsn.capabilities.peerkey,wlan_mgt.tcprep.trsmt_pow,wlan_mgt.tcprep.link_mrg,wlan.wep.iv,wlan.wep.key,wlan.wep.icv,wlan.tkip.extiv,wlan.ccmp.extiv,wlan.qos.tid,wlan.qos.priority,wlan.qos.eosp,wlan.qos.ack,wlan.qos.amsdupresent,wlan.qos.buf_state_indicated,wlan.qos.bit4,wlan.qos.txop_dur_req,wlan.qos.buf_state_indicated.1,data.len
0,0,0,0,0.000066,0.000066,0.009150,0.009150,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.36865,0,0,0,0,...,0,0,0.0,0,0.000000,0.00000,0.000000,0.000000,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0
1,0,0,0,0.000014,0.000014,0.000000,0.000000,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.36867,0,0,0,0,...,0,0,0.0,0,0.000000,0.00000,0.000000,0.000000,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0
2,0,0,0,0.035528,0.035528,0.070588,0.070588,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.36871,0,0,0,0,...,0,0,0.0,1,0.001459,0.46154,0.000000,0.007936,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0
3,0,0,0,0.005128,0.005128,0.094771,0.094771,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.36876,0,0,0,0,...,0,0,0.0,1,0.000875,0.46154,0.000000,0.003968,0,0,0.0,0.000015,0.007843,0.000031,0.000016,1,0,0,0,0,0,0,0,0.5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0
4,0,0,0,0.035116,0.035116,0.070588,0.070588,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.36880,0,0,0,0,...,0,0,0.0,1,0.001459,0.46154,0.004566,0.007936,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0


In [ ]:
print(x_testing.shape)
x_testing.head()

(40158, 152)


,frame.interface_id,frame.dlt,frame.offset_shift,frame.time_delta,frame.time_delta_displayed,frame.len,frame.cap_len,frame.marked,frame.ignored,radiotap.version,radiotap.pad,radiotap.length,radiotap.present.tsft,radiotap.present.flags,radiotap.present.rate,radiotap.present.channel,radiotap.present.fhss,radiotap.present.dbm_antsignal,radiotap.present.dbm_antnoise,radiotap.present.lock_quality,radiotap.present.tx_attenuation,radiotap.present.db_tx_attenuation,radiotap.present.dbm_tx_power,radiotap.present.antenna,radiotap.present.db_antsignal,radiotap.present.db_antnoise,radiotap.present.rxflags,radiotap.present.xchannel,radiotap.present.mcs,radiotap.present.ampdu,radiotap.present.vht,radiotap.present.reserved,radiotap.present.rtap_ns,radiotap.present.vendor_ns,radiotap.present.ext,radiotap.mactime,radiotap.flags.cfp,radiotap.flags.preamble,radiotap.flags.wep,radiotap.flags.frag,...,wlan_mgt.fixed.chanwidth,wlan_mgt.fixed.fragment,wlan_mgt.fixed.sequence,wlan_mgt.tagged.all,wlan_mgt.ssid,wlan_mgt.ds.current_channel,wlan_mgt.tim.dtim_count,wlan_mgt.tim.dtim_period,wlan_mgt.tim.bmapctl.multicast,wlan_mgt.tim.bmapctl.offset,wlan_mgt.country_info.environment,wlan_mgt.rsn.version,wlan_mgt.rsn.gcs.type,wlan_mgt.rsn.pcs.count,wlan_mgt.rsn.akms.count,wlan_mgt.rsn.akms.type,wlan_mgt.rsn.capabilities.preauth,wlan_mgt.rsn.capabilities.no_pairwise,wlan_mgt.rsn.capabilities.ptksa_replay_counter,wlan_mgt.rsn.capabilities.gtksa_replay_counter,wlan_mgt.rsn.capabilities.mfpr,wlan_mgt.rsn.capabilities.mfpc,wlan_mgt.rsn.capabilities.peerkey,wlan_mgt.tcprep.trsmt_pow,wlan_mgt.tcprep.link_mrg,wlan.wep.iv,wlan.wep.key,wlan.wep.icv,wlan.tkip.extiv,wlan.ccmp.extiv,wlan.qos.tid,wlan.qos.priority,wlan.qos.eosp,wlan.qos.ack,wlan.qos.amsdupresent,wlan.qos.buf_state_indicated,wlan.qos.bit4,wlan.qos.txop_dur_req,wlan.qos.buf_state_indicated.1,data.len
0,0,0,0,0.002547,0.002547,0.98824,0.98824,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.88219,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.22215,0.0,0.47729,0.0,0.0,0.16667,0.16667,0,0,0,0,0,0,0,0.98674
1,0,0,0,0.003296,0.003296,0.98824,0.98824,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.88220,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.35106,0.0,0.18516,0.0,0.0,0.16667,0.16667,0,0,0,0,0,0,0,0.98674
2,0,0,0,0.003285,0.003285,0.98824,0.98824,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.88220,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.43700,0.0,0.47541,0.0,0.0,0.16667,0.16667,0,0,0,0,0,0,0,0.98674
3,0,0,0,0.005942,0.005942,0.00000,0.00000,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.88220,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.00000,0.0,0.00000,0.0,0.0,0.00000,0.00000,0,0,0,0,0,0,0,0.00000
4,0,0,0,0.001519,0.001519,0.98824,0.98824,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.88220,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.63621,0.0,0.89971,0.0,0.0,0.16667,0.16667,0,0,0,0,0,0,0,0.98674



###**Constructing and Selecting Features**

####**Feature Extraction using Autoencoder**


In [ ]:
x_training.isnull().any() 

frame.interface_id                False
frame.dlt                         False
frame.offset_shift                False
frame.time_delta                  False
frame.time_delta_displayed        False
                                  ...  
wlan.qos.buf_state_indicated      False
wlan.qos.bit4                     False
wlan.qos.txop_dur_req             False
wlan.qos.buf_state_indicated.1    False
data.len                          False
Length: 152, dtype: bool

In [ ]:
# Rescaling using MinMaxScaler prior to running the Sparse Autoencoders
from sklearn.preprocessing import MinMaxScaler
from numpy import set_printoptions

scaler = MinMaxScaler(feature_range=(0, 1))

x_training = scaler.fit_transform(x_training)
training = pd.DataFrame(x_training, columns = list(training.columns.values)[0:152])

x_testing = scaler.fit_transform(x_testing)
testing= pd.DataFrame(x_training, columns = list(testing.columns.values)[0:152])


In [ ]:
x_training

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
x_testing

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.98674  ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.98674  ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.98674  ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.0026525],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.0026525],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.0026525]])

In [ ]:
import numpy as np

# Reshape input as vector
x_training = x_training.reshape((len(x_training), np.prod(x_training.shape[1:])))
x_testing = x_testing.reshape((len(x_testing), np.prod(x_testing.shape[1:])))
print(x_training.shape)
print(x_testing.shape)

(97044, 152)
(40158, 152)


In [ ]:
# Sparse Autoencoders with L1 regularization
import keras
from keras.regularizers import l1 
from keras.optimizers import Adam
from keras.layers import Dense, Input 
import tensorflow as tf

tf.random.set_seed(100)

input_size = 152 
code_size = 20

input = keras.Input(shape=(input_size,))
# Activity regularisation with L1 norm
code = Dense(code_size, activation='relu', activity_regularizer=l1(10e-6))(input) 
output = Dense(input_size, activation='sigmoid')(code)

autoencoder_regularized = keras.Model(input, output) 
autoencoder_regularized.compile(optimizer='adam', loss='binary_crossentropy')
history_regularized = autoencoder_regularized.fit(x_training, x_training, epochs=20)

encoded_regularized = keras.Model(input, code)

Epoch 1/20
3033/3033 [==============================] - 7s 2ms/step - loss: 0.1094
Epoch 2/20
3033/3033 [==============================] - 7s 2ms/step - loss: 0.0303
Epoch 3/20
3033/3033 [==============================] - 6s 2ms/step - loss: 0.0285
Epoch 4/20
3033/3033 [==============================] - 6s 2ms/step - loss: 0.0278
Epoch 5/20
3033/3033 [==============================] - 6s 2ms/step - loss: 0.0276
Epoch 6/20
3033/3033 [==============================] - 6s 2ms/step - loss: 0.0275
Epoch 7/20
3033/3033 [==============================] - 6s 2ms/step - loss: 0.0274
Epoch 8/20
3033/3033 [==============================] - 7s 2ms/step - loss: 0.0274
Epoch 9/20
3033/3033 [==============================] - 7s 2ms/step - loss: 0.0273
Epoch 10/20
3033/3033 [==============================] - 6s 2ms/step - loss: 0.0273
Epoch 11/20
3033/3033 [==============================] - 6s 2ms/step - loss: 0.0273
Epoch 12/20
3033/3033 [==============================] - 6s 2ms/step - loss: 0.0273
E

In [ ]:
x_test_encoded = encoded_regularized.predict(x_testing)

In [ ]:
x_training_encoded = encoded_regularized.predict(x_training)

In [ ]:
x_test_encoded.shape

(40158, 20)

In [ ]:
x_testing.shape

(40158, 152)

In [ ]:
x_training_encoded.shape

(97044, 20)

In [ ]:
x_training.shape

(97044, 152)

####**Combining Original dataset and new extracted features from the autoencoder**



In [ ]:
X_training = np.concatenate((x_training, x_training_encoded), axis = 1)

In [ ]:
X_training.shape

(97044, 172)

In [ ]:
X_testing = np.concatenate((x_testing, x_test_encoded), axis = 1)

In [ ]:
X_testing.shape

(40158, 172)

####**Feature Selection step 1**


In [ ]:
# Checking X_training type prior to selection process
type(X_training)

numpy.ndarray

In [ ]:
X_training.shape

(97044, 172)

In [ ]:
# Changing X_training type from Numpy array to Pandas DataFrame
import pandas as pd

X_training = pd.DataFrame (data=X_training[0:,0:],
                      index= [i for i in range(X_training.shape[0])],
                      columns=[str(i) for i in range (X_training.shape[1])])
X_training.shape

(97044, 172)

In [ ]:
X_training.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171
0,0.0,0.0,0.0,0.000064,0.000064,0.009150,0.009150,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.394895,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.138364,0.493176,1.684572,1.285296,3.663028,0.0,0.775278,2.934880,0.152947,0.365913,0.0,0.696687,1.255560,0.0,1.261514,0.393823,0.000000,0.000000,0.0,3.552652
1,0.0,0.0,0.0,0.000012,0.000012,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.394916,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.936183,1.545995,0.525187,1.118178,2.627026,0.0,1.332971,0.655273,1.027668,0.606306,0.0,0.000000,3.525524,0.0,0.350903,0.863805,0.000000,0.000000,0.0,2.885817
2,0.0,0.0,0.0,0.036308,0.036308,0.070588,0.070588,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.394959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.756648,1.081140,0.000000,0.964040,1.659521,0.0,2.053579,1.190628,0.518096,6.248455,0.0,1.950850,5.748194,0.0,0.658739,1.302463,1.161693,0.405554,0.0,0.000000
3,0.0,0.0,0.0,0.005238,0.005238,0.094771,0.094771,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.395013,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.193164,3.684281,0.028062,0.868532,5.088230,0.0,1.022224,2.898563,0.423081,6.701244,0.0,0.591442,6.216964,0.0,2.750768,0.000000,0.745862,1.077770,0.0,0.000000
4,0.0,0.0,0.0,0.035887,0.035887,0.070588,0.070588,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.395055,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.421315,0.488398,0.000000,0.848438,1.523081,0.0,2.441001,0.434584,0.723999,6.805610,0.0,2.056355,5.691250,0.0,0.425010,1.221536,1.369457,0.000000,0.0,0.000000


In [ ]:
type(X_training) 

pandas.core.frame.DataFrame

In [ ]:
type(X_testing)

numpy.ndarray

In [ ]:
# Changing X_testing type from Numpy array to Pandas DataFrame
import pandas as pd

X_testing = pd.DataFrame (data=X_testing[0:,0:],
                      index= [i for i in range(X_testing.shape[0])],
                      columns=['Column '+ str(i) for i in range (X_testing.shape[1])])
X_testing.shape

(40158, 172)

In [ ]:
type(X_testing)

pandas.core.frame.DataFrame

In [ ]:
X_testing.head()

,Column 0,Column 1,Column 2,Column 3,Column 4,Column 5,Column 6,Column 7,Column 8,Column 9,Column 10,Column 11,Column 12,Column 13,Column 14,Column 15,Column 16,Column 17,Column 18,Column 19,Column 20,Column 21,Column 22,Column 23,Column 24,Column 25,Column 26,Column 27,Column 28,Column 29,Column 30,Column 31,Column 32,Column 33,Column 34,Column 35,Column 36,Column 37,Column 38,Column 39,...,Column 132,Column 133,Column 134,Column 135,Column 136,Column 137,Column 138,Column 139,Column 140,Column 141,Column 142,Column 143,Column 144,Column 145,Column 146,Column 147,Column 148,Column 149,Column 150,Column 151,Column 152,Column 153,Column 154,Column 155,Column 156,Column 157,Column 158,Column 159,Column 160,Column 161,Column 162,Column 163,Column 164,Column 165,Column 166,Column 167,Column 168,Column 169,Column 170,Column 171
0,0.0,0.0,0.0,0.002809,0.002809,0.98824,0.98824,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.901749,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.222181,0.0,0.477300,0.0,0.0,0.16667,0.16667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.98674,0.900261,0.429014,0.856608,0.000000,1.198432,0.0,1.552701,1.403326,1.485049,0.000000,0.0,0.788850,0.039085,0.0,0.529108,0.889999,0.051608,2.873188,0.0,0.929745
1,0.0,0.0,0.0,0.003637,0.003637,0.98824,0.98824,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.901759,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.351109,0.0,0.185164,0.0,0.0,0.16667,0.16667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.98674,0.896495,0.523902,0.868367,0.000000,1.450363,0.0,1.533947,1.329271,1.456053,0.000000,0.0,0.697959,0.000000,0.0,0.785076,0.627621,0.116546,2.885051,0.0,0.769502
2,0.0,0.0,0.0,0.003624,0.003624,0.98824,0.98824,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.901759,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.437061,0.0,0.475420,0.0,0.0,0.16667,0.16667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.98674,0.853779,0.348136,0.851271,0.000000,1.261769,0.0,1.535358,1.164427,1.465730,0.000000,0.0,0.830441,0.000000,0.0,0.690914,0.726563,0.131993,2.946114,0.0,0.909897
3,0.0,0.0,0.0,0.006561,0.006561,0.00000,0.00000,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.901759,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,1.099770,1.373006,0.835287,0.220531,2.972025,0.0,1.467718,0.466146,1.229540,0.442589,0.0,0.023981,3.673919,0.0,0.358298,0.959236,0.000000,0.000000,0.0,2.791876
4,0.0,0.0,0.0,0.001672,0.001672,0.98824,0.98824,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.901759,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.636299,0.0,0.899728,0.0,0.0,0.16667,0.16667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.98674,0.812163,1.127965,0.000000,0.000000,0.628896,0.0,0.757731,1.117347,1.636739,0.000000,0.0,1.467792,0.000000,0.0,1.152414,1.049292,1.403978,3.347192,0.0,1.573107


In [ ]:
# Adding header to the combined dataset that include original and extracted features
headername = 'headers_without4and7_withExtractedFeatures.csv'
headers = pd.read_csv(headername)

X_training.columns = headers.columns
X_testing.columns = headers.columns



In [ ]:
X_training.head()

,frame.interface_id,frame.dlt,frame.offset_shift,frame.time_delta,frame.time_delta_displayed,frame.len,frame.cap_len,frame.marked,frame.ignored,radiotap.version,radiotap.pad,radiotap.length,radiotap.present.tsft,radiotap.present.flags,radiotap.present.rate,radiotap.present.channel,radiotap.present.fhss,radiotap.present.dbm_antsignal,radiotap.present.dbm_antnoise,radiotap.present.lock_quality,radiotap.present.tx_attenuation,radiotap.present.db_tx_attenuation,radiotap.present.dbm_tx_power,radiotap.present.antenna,radiotap.present.db_antsignal,radiotap.present.db_antnoise,radiotap.present.rxflags,radiotap.present.xchannel,radiotap.present.mcs,radiotap.present.ampdu,radiotap.present.vht,radiotap.present.reserved,radiotap.present.rtap_ns,radiotap.present.vendor_ns,radiotap.present.ext,radiotap.mactime,radiotap.flags.cfp,radiotap.flags.preamble,radiotap.flags.wep,radiotap.flags.frag,...,wlan_mgt.rsn.capabilities.mfpr,wlan_mgt.rsn.capabilities.mfpc,wlan_mgt.rsn.capabilities.peerkey,wlan_mgt.tcprep.trsmt_pow,wlan_mgt.tcprep.link_mrg,wlan.wep.iv,wlan.wep.key,wlan.wep.icv,wlan.tkip.extiv,wlan.ccmp.extiv,wlan.qos.tid,wlan.qos.priority,wlan.qos.eosp,wlan.qos.ack,wlan.qos.amsdupresent,wlan.qos.buf_state_indicated,wlan.qos.bit4,wlan.qos.txop_dur_req,wlan.qos.buf_state_indicated.1,data.len,Extracted 1,Extracted 2,Extracted 3,Extracted 4,Extracted 5,Extracted 6,Extracted 7,Extracted 8,Extracted 9,Extracted 10,Extracted 11,Extracted 12,Extracted 13,Extracted 14,Extracted 15,Extracted 16,Extracted 17,Extracted 18,Extracted 19,Extracted 20
0,0.0,0.0,0.0,0.000064,0.000064,0.009150,0.009150,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.394895,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.138364,0.493176,1.684572,1.285296,3.663028,0.0,0.775278,2.934880,0.152947,0.365913,0.0,0.696687,1.255560,0.0,1.261514,0.393823,0.000000,0.000000,0.0,3.552652
1,0.0,0.0,0.0,0.000012,0.000012,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.394916,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.936183,1.545995,0.525187,1.118178,2.627026,0.0,1.332971,0.655273,1.027668,0.606306,0.0,0.000000,3.525524,0.0,0.350903,0.863805,0.000000,0.000000,0.0,2.885817
2,0.0,0.0,0.0,0.036308,0.036308,0.070588,0.070588,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.394959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.756648,1.081140,0.000000,0.964040,1.659521,0.0,2.053579,1.190628,0.518096,6.248455,0.0,1.950850,5.748194,0.0,0.658739,1.302463,1.161693,0.405554,0.0,0.000000
3,0.0,0.0,0.0,0.005238,0.005238,0.094771,0.094771,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.395013,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.193164,3.684281,0.028062,0.868532,5.088230,0.0,1.022224,2.898563,0.423081,6.701244,0.0,0.591442,6.216964,0.0,2.750768,0.000000,0.745862,1.077770,0.0,0.000000
4,0.0,0.0,0.0,0.035887,0.035887,0.070588,0.070588,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.395055,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.421315,0.488398,0.000000,0.848438,1.523081,0.0,2.441001,0.434584,0.723999,6.805610,0.0,2.056355,5.691250,0.0,0.425010,1.221536,1.369457,0.000000,0.0,0.000000


In [ ]:
X_testing.head()

,frame.interface_id,frame.dlt,frame.offset_shift,frame.time_delta,frame.time_delta_displayed,frame.len,frame.cap_len,frame.marked,frame.ignored,radiotap.version,radiotap.pad,radiotap.length,radiotap.present.tsft,radiotap.present.flags,radiotap.present.rate,radiotap.present.channel,radiotap.present.fhss,radiotap.present.dbm_antsignal,radiotap.present.dbm_antnoise,radiotap.present.lock_quality,radiotap.present.tx_attenuation,radiotap.present.db_tx_attenuation,radiotap.present.dbm_tx_power,radiotap.present.antenna,radiotap.present.db_antsignal,radiotap.present.db_antnoise,radiotap.present.rxflags,radiotap.present.xchannel,radiotap.present.mcs,radiotap.present.ampdu,radiotap.present.vht,radiotap.present.reserved,radiotap.present.rtap_ns,radiotap.present.vendor_ns,radiotap.present.ext,radiotap.mactime,radiotap.flags.cfp,radiotap.flags.preamble,radiotap.flags.wep,radiotap.flags.frag,...,wlan_mgt.rsn.capabilities.mfpr,wlan_mgt.rsn.capabilities.mfpc,wlan_mgt.rsn.capabilities.peerkey,wlan_mgt.tcprep.trsmt_pow,wlan_mgt.tcprep.link_mrg,wlan.wep.iv,wlan.wep.key,wlan.wep.icv,wlan.tkip.extiv,wlan.ccmp.extiv,wlan.qos.tid,wlan.qos.priority,wlan.qos.eosp,wlan.qos.ack,wlan.qos.amsdupresent,wlan.qos.buf_state_indicated,wlan.qos.bit4,wlan.qos.txop_dur_req,wlan.qos.buf_state_indicated.1,data.len,Extracted 1,Extracted 2,Extracted 3,Extracted 4,Extracted 5,Extracted 6,Extracted 7,Extracted 8,Extracted 9,Extracted 10,Extracted 11,Extracted 12,Extracted 13,Extracted 14,Extracted 15,Extracted 16,Extracted 17,Extracted 18,Extracted 19,Extracted 20
0,0.0,0.0,0.0,0.002809,0.002809,0.98824,0.98824,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.901749,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.222181,0.0,0.477300,0.0,0.0,0.16667,0.16667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.98674,0.900261,0.429014,0.856608,0.000000,1.198432,0.0,1.552701,1.403326,1.485049,0.000000,0.0,0.788850,0.039085,0.0,0.529108,0.889999,0.051608,2.873188,0.0,0.929745
1,0.0,0.0,0.0,0.003637,0.003637,0.98824,0.98824,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.901759,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.351109,0.0,0.185164,0.0,0.0,0.16667,0.16667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.98674,0.896495,0.523902,0.868367,0.000000,1.450363,0.0,1.533947,1.329271,1.456053,0.000000,0.0,0.697959,0.000000,0.0,0.785076,0.627621,0.116546,2.885051,0.0,0.769502
2,0.0,0.0,0.0,0.003624,0.003624,0.98824,0.98824,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.901759,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.437061,0.0,0.475420,0.0,0.0,0.16667,0.16667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.98674,0.853779,0.348136,0.851271,0.000000,1.261769,0.0,1.535358,1.164427,1.465730,0.000000,0.0,0.830441,0.000000,0.0,0.690914,0.726563,0.131993,2.946114,0.0,0.909897
3,0.0,0.0,0.0,0.006561,0.006561,0.00000,0.00000,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.901759,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,1.099770,1.373006,0.835287,0.220531,2.972025,0.0,1.467718,0.466146,1.229540,0.442589,0.0,0.023981,3.673919,0.0,0.358298,0.959236,0.000000,0.000000,0.0,2.791876
4,0.0,0.0,0.0,0.001672,0.001672,0.98824,0.98824,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.901759,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.636299,0.0,0.899728,0.0,0.0,0.16667,0.16667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.98674,0.812163,1.127965,0.000000,0.000000,0.628896,0.0,0.757731,1.117347,1.636739,0.000000,0.0,1.467792,0.000000,0.0,1.152414,1.049292,1.403978,3.347192,0.0,1.573107


In [ ]:
# Saving dataset with combined original and extracted features
X_training.to_csv('full_dataset_training.csv' , sep=',' , index=False)
X_testing.to_csv('full_dataset_testing.csv' , sep=',' , index=False)


Now that he dataset is ready for the feature selection, we can proceed to step 1 of Feature Selection.
The 1st step is to remove all features were 80% of the datapoints have the same value.
Using threshold = 0.8 * (1 - 0.8).
This is motivated by the fact that those features would have low variance thus containing less valuable information for the classification models. 

In [ ]:
from sklearn.feature_selection import VarianceThreshold
random.seed(100)

# Applying VarianceThreshold with get.support function to retain the pandas dataframe format [1].
def Variance_Threshold_DataFrame(data, threshold=(0.16)):
    selector = VarianceThreshold(threshold= (0.16))
    selector.fit(X_training)
    return data[X_training.columns[selector.get_support(indices=True)]]
X_training = Variance_Threshold_DataFrame(X_training, 0.16)


In [ ]:
random_seed = 100
X_training.shape

(97044, 20)

In [ ]:
X_training.head()

,radiotap.datarate,radiotap.channel.type.cck,radiotap.channel.type.ofdm,wlan.fc.ds,wlan.fc.pwrmgt,wlan.fc.protected,Extracted 2,Extracted 3,Extracted 5,Extracted 7,Extracted 8,Extracted 9,Extracted 10,Extracted 12,Extracted 13,Extracted 15,Extracted 16,Extracted 17,Extracted 18,Extracted 20
0,0.88679,0.0,1.0,0.5,1.0,0.0,0.493176,1.684572,3.663028,0.775278,2.934880,0.152947,0.365913,0.696687,1.255560,1.261514,0.393823,0.000000,0.000000,3.552652
1,0.43396,0.0,1.0,0.0,0.0,0.0,1.545995,0.525187,2.627026,1.332971,0.655273,1.027668,0.606306,0.000000,3.525524,0.350903,0.863805,0.000000,0.000000,2.885817
2,0.00000,1.0,0.0,0.0,0.0,0.0,1.081140,0.000000,1.659521,2.053579,1.190628,0.518096,6.248455,1.950850,5.748194,0.658739,1.302463,1.161693,0.405554,0.000000
3,0.00000,1.0,0.0,0.0,0.0,0.0,3.684281,0.028062,5.088230,1.022224,2.898563,0.423081,6.701244,0.591442,6.216964,2.750768,0.000000,0.745862,1.077770,0.000000
4,0.00000,1.0,0.0,0.0,0.0,0.0,0.488398,0.000000,1.523081,2.441001,0.434584,0.723999,6.805610,2.056355,5.691250,0.425010,1.221536,1.369457,0.000000,0.000000


Original features columns to keep after VarianceThreshold has ben used:
 6 of the original features can be kept after Variancethreshold has been used.


Extracted Features to keep after using VarianceThreshold:
14 extracted features can be kept after VArianceThreshold has been run


In [ ]:
random_seed = 100
X_training.shape

(97044, 20)

In [ ]:
# Saving X_training dataset after feature selection step 1
X_training.to_csv('dataset_variance_threshold_selection.csv' , sep=',' , index=False)

####**Feature Selection step 2**


In [ ]:
''' Recursive Feature Elimination (RFE class) wrapper method.
This method measures the usefulness of the features based on the classifier performance 
(classifier used here: Logistic Regression) but this is 
computationaly expensive, so the dataset was reduced using VarianceThreshold first. '''


from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
random_seed=100

# feature extraction 
model = LogisticRegression()
rfe = RFE(model, 5)
fit = rfe.fit(X_training, y_training)
print("Num Features: %d" % fit.n_features_)
print("selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Num Features: 5
selected Features: [ True  True False False False False False False  True False False False
 False  True False False False False False  True]
Feature Ranking: [ 1  1  3  2 14  6 12  8  1  4 11  9 10  1  7 16 15 13  5  1]


In [ ]:
# X_training = X_training[['wlan.fc.ds','wlan.fc.protected', 'Extracted 10','Extracted 11', 'Extracted 12']]
X_training = X_training[['radiotap.datarate','radiotap.channel.type.cck', 'Extracted 5','Extracted 12', 'Extracted 20']]

print(X_training.shape)
X_training.head()

(97044, 5)


,radiotap.datarate,radiotap.channel.type.cck,Extracted 5,Extracted 12,Extracted 20
0,0.88679,0.0,3.663028,0.696687,3.552652
1,0.43396,0.0,2.627026,0.000000,2.885817
2,0.00000,1.0,1.659521,1.950850,0.000000
3,0.00000,1.0,5.088230,0.591442,0.000000
4,0.00000,1.0,1.523081,2.056355,0.000000


In [ ]:
# Keep the same column from testing dataset
X_testing = X_testing[['radiotap.datarate','radiotap.channel.type.cck', 'Extracted 5','Extracted 12', 'Extracted 20']]

print(X_testing.shape)
X_testing.head()

(40158, 5)


,radiotap.datarate,radiotap.channel.type.cck,Extracted 5,Extracted 12,Extracted 20
0,1.00000,0.0,1.198432,0.788850,0.929745
1,1.00000,0.0,1.450363,0.697959,0.769502
2,1.00000,0.0,1.261769,0.830441,0.909897
3,0.43396,0.0,2.972025,0.023981,2.791876
4,1.00000,0.0,0.628896,1.467792,1.573107



###**Building Machine Learning algorithms for detection task**

ML algorithms that I am going to try

> Logistic Regression as base algorithm
> SVM with linear kernel
> SVM with RBF kernel
> SVM with Polynomial kernel
> SVM with sigmoid kernel



####**Machine Learning Algoritm 1**





In [ ]:
# Logistic Regression as base algorithm
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics

LR = LogisticRegression()
model.LR = LR.fit(X_training, y_training)
predictionsLR = model.LR.predict(X_testing)
CMLogisticRegression = metrics.confusion_matrix(y_testing, predictionsLR)
print(CMLogisticRegression)

[[19603   476]
 [ 2428 17651]]


####**Machine Learning Algorithm 2**

In [ ]:
# SVM with linear kernel
from sklearn.svm import SVC
import sklearn.metrics as metrics

svc = SVC(kernel = 'linear')
modelLinearSVM = svc.fit(X_training, y_training)
predictionsLinearSVM = modelLinearSVM.predict(X_testing)
CMLinearSVM = metrics.confusion_matrix(y_testing, predictionsLinearSVM)
print(CMLinearSVM)

[[19675   404]
 [ 3886 16193]]



####**Machine Learning Algorithm 3**


In [ ]:
# SVM with polynomial kernel
from sklearn.svm import SVC
svc = SVC(kernel = 'poly', degree = 3)
modelPolySVM = svc.fit(X_training, y_training)
predictionsPolySVM = modelPolySVM.predict(X_testing)
CMPolySVM = metrics.confusion_matrix(y_testing, predictionsPolySVM)
print(CMPolySVM)

[[19640   439]
 [ 3997 16082]]


####**Machine Learning Algorithm 4**


In [ ]:
# SVM with RBF kernel (RBF)
from sklearn.svm import SVC
svc = SVC(kernel = 'rbf')
modelRBFSVM = svc.fit(X_training, y_training)
predictionsRBFSVM = modelRBFSVM.predict(X_testing)
CMRBFSVM = metrics.confusion_matrix(y_testing, predictionsRBFSVM)
print(CMRBFSVM)

[[19684   395]
 [ 4668 15411]]


####**Machine Learning Algorithm 5**


In [ ]:
# SVM with Sigmoid Kernel

from sklearn.svm import SVC
svc = SVC(kernel = 'sigmoid')
modelSigSVM = svc.fit(X_training, y_training)
predictionsSigSVM = modelSigSVM.predict(X_testing)
CMSigSVM = metrics.confusion_matrix(y_testing, predictionsRBFSVM)
print(CMSigSVM)

[[19684   395]
 [ 4668 15411]]



###**Evaluating models and Analysing results to select the best models**

In [ ]:
# ML model 1: Logistic Regression Evaluation
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_testing, predictionsLR))
print(classification_report(y_testing, predictionsLR))

[[19603   476]
 [ 2428 17651]]
              precision    recall  f1-score   support

           0       0.89      0.98      0.93     20079
           1       0.97      0.88      0.92     20079

    accuracy                           0.93     40158
   macro avg       0.93      0.93      0.93     40158
weighted avg       0.93      0.93      0.93     40158



In [ ]:
# Calculate mcc for LR
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_testing, predictionsLR)


0.8594421973989401

In [ ]:
# ML mode 2: SVM with Linear kernel Evaluation
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_testing, predictionsLinearSVM))
print(classification_report(y_testing, predictionsLinearSVM))

[[19675   404]
 [ 3886 16193]]
              precision    recall  f1-score   support

           0       0.84      0.98      0.90     20079
           1       0.98      0.81      0.88     20079

    accuracy                           0.89     40158
   macro avg       0.91      0.89      0.89     40158
weighted avg       0.91      0.89      0.89     40158



In [ ]:
# Calculate mcc for SVM Linear kernel
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_testing, predictionsLinearSVM)

0.7984412540769469

In [ ]:
# ML model 3: SVM with polynomial kernel Evaluation
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_testing,predictionsPolySVM))
print(classification_report(y_testing, predictionsPolySVM))

[[19640   439]
 [ 3997 16082]]
              precision    recall  f1-score   support

           0       0.83      0.98      0.90     20079
           1       0.97      0.80      0.88     20079

    accuracy                           0.89     40158
   macro avg       0.90      0.89      0.89     40158
weighted avg       0.90      0.89      0.89     40158



In [ ]:
# Calculate mcc for SVM poly kernel
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_testing, predictionsPolySVM)

0.7915998479283526

In [ ]:
# ML model 4: SVM with RBF kernel Evaluation
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_testing,predictionsRBFSVM))
print(classification_report(y_testing, predictionsRBFSVM))

[[19684   395]
 [ 4668 15411]]
              precision    recall  f1-score   support

           0       0.81      0.98      0.89     20079
           1       0.98      0.77      0.86     20079

    accuracy                           0.87     40158
   macro avg       0.89      0.87      0.87     40158
weighted avg       0.89      0.87      0.87     40158



In [ ]:
# Calculate mcc for SVM rbf kernel
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_testing, predictionsRBFSVM)

0.7653779621885266

In [ ]:
# ML model 5: SVM with Sigmoid Kernel
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_testing,predictionsSigSVM))
print(classification_report(y_testing, predictionsSigSVM))

[[14708  5371]
 [ 1816 18263]]
              precision    recall  f1-score   support

           0       0.89      0.73      0.80     20079
           1       0.77      0.91      0.84     20079

    accuracy                           0.82     40158
   macro avg       0.83      0.82      0.82     40158
weighted avg       0.83      0.82      0.82     40158



In [ ]:
# Calculate mcc for SVM sigmoid kernel
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_testing, predictionsSigSVM)

0.6523701604366937

References:

[1] https://stackoverflow.com/questions/39812885/retain-feature-names-after-scikit-feature-selection
